### ANÁLISE DE SENTIMENTO EM REVIEWS DE TRÊS ESTRELAS: UMA PROPOSTA DE CLASSIFICAÇÃO MULTIRRÓTULO

##### Seção 7. Preparação dos Dados para os Modelos de Aprendizado de Máquina

#### Importação das Bibliotecas

In [1]:
import pandas as pd
import re
import nltk
import numpy as np
nltk.download('stopwords')
from nltk.corpus import stopwords
import spacy
import spacy.cli
spacy.cli.download("pt_core_news_sm")
import pt_core_news_sm
spc_pt = pt_core_news_sm.load()
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\carolina.gadelha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


#### Task 0: Carregar o dataset B2W-Reviews01-Anotado

In [2]:
#Carrega o dataset anotado
data = pd.read_csv('../Documents/TCC/b2w-reviews-01-anotado.csv')

#### Task 1: Eliminar Reviews Neutras

In [3]:
data = data[data['anotacao'] != 'neutro']
data.reset_index(drop=True, inplace=True)

In [4]:
data.anotacao.value_counts()

positivo  negativo    2971
positivo              1283
negativo               525
Name: anotacao, dtype: int64

#### Task 2: Preparação dos Dados para Modelos

In [5]:
#Remove da lista de stopwords palavras importantes
stopwords_pt = stopwords.words("portuguese")
stopwords_pt = [i for i in stopwords_pt if i not in ["não", "nenhum", "nada", "jamais", "nunca", "nem"]]


def limpa_texto(texto):
    texto = texto.lower() #Transforma texto em minúsculo
    texto = re.sub(r"[\W\d_]+", " ", texto) #Filtra apenas letras
    texto = [pal for pal in texto.split() if pal not in stopwords_pt] #Remove stopwords
    spc_texto = spc_pt(" ".join(texto))
    tokens = [word.lemma_ if word.lemma_ != "-PRON-" else word.lower_ for word in spc_texto] #Lemmatiza

    return " ".join(tokens)

#Aplica função na coluna Review
data['review'] = data['review'].apply(limpa_texto)


#### Task 3: Feature extraction

Com Bag of Words

In [6]:
# Instancia o CountVectorizer
vectorizer = CountVectorizer(binary=True)

# Vetoriza a coluna Review
texto = data['review']
X_bow = vectorizer.fit_transform(texto)

#Transforma o resultado em Array
X_bow.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

Com TF-IDF

In [7]:
# Instancia o TfidfVectorizer
tfidf_vect = TfidfVectorizer(max_features=5000)

# Vetoriza o texto
X_tfidf = tfidf_vect.fit_transform(texto)

print(X_tfidf)

  (0, 452)	0.38296472883147425
  (0, 800)	0.20241106165447742
  (0, 773)	0.3236313282657797
  (0, 439)	0.22820142570212948
  (0, 980)	0.17255173514950065
  (0, 4468)	0.396589936528068
  (0, 4572)	0.2588270025734173
  (0, 2942)	0.20650831523815913
  (0, 2764)	0.2339493225690672
  (0, 4110)	0.23051222704853883
  (0, 1868)	0.21525769785794982
  (0, 476)	0.07313409329178633
  (0, 3178)	0.07884290235156338
  (0, 844)	0.27889970674955683
  (0, 95)	0.3395673373087828
  (1, 98)	0.18465659203197232
  (1, 3969)	0.18641184799664837
  (1, 4274)	0.2828496912062698
  (1, 4318)	0.17216637862563597
  (1, 998)	0.16113141204735434
  (1, 4064)	0.3428957586272838
  (1, 4122)	0.32197760106459355
  (1, 123)	0.3428957586272838
  (1, 1556)	0.22935907399355002
  (1, 3957)	0.2447026594529863
  :	:
  (4776, 1605)	0.1904493852418518
  (4776, 138)	0.12441186355061533
  (4776, 3937)	0.22620230027385466
  (4776, 1649)	0.15969146216534708
  (4776, 2448)	0.0575783480004296
  (4776, 3178)	0.050032083500091945
  (4777, 

#### Task 4: Binarizar as Classes de Sentimento

In [8]:
#Lista as classes de sentimento existentes no dataset
SENTIMENTOS = ['positivo','negativo']

#Transforma as classes em coluna com valores preenchidos 
for sentimento in SENTIMENTOS[::-1]:
    data.insert(1,sentimento,data.anotacao.apply(lambda x: 1 if sentimento in x else 0))

#Visualização
data.head()

,reviewer_id,positivo,negativo,review,anotacao,recommend_to_a_friend,site_category_lv1,reviewer_birth_year,reviewer_gender,overall_rating
0,c2b3ebfc9238e1462555a7e36539a078c6fb85e5438e39...,1,1,agilidade configuração produto bom imagem som ...,positivo negativo,Yes,Informática,1899.0,M,3
1,1345ab31c9fa498aac8dd78b1c1069a056f237e20fdba7...,1,1,atender expectativa outro fritadeira outro mar...,positivo negativo,Yes,Eletroportáteis,1900.0,F,3
2,b68330385cc203b6e8249f796d592ae6173df1adda5ae3...,1,1,bom filme filme assistir curtir famílio luta v...,positivo negativo,Yes,Filmes e Séries,1906.0,M,3
3,ed921e3d758d33b09938bb46d932c3cdbef9f821b68a0e...,1,0,produto bom acabamento recomendar produto pois...,positivo,Yes,Móveis,1908.0,M,3
4,dd8279ea157b3802bb5b68a2fb647595178e941f889386...,1,0,preço ta otimo calça real show bola bom econom...,positivo,Yes,Moda,1924.0,F,3


In [9]:
# Converte os valores para Float
data['positivo'] = data['positivo'].astype(float)
data['negativo'] = data['negativo'].astype(float)
data.dtypes

reviewer_id               object
positivo                 float64
negativo                 float64
review                    object
anotacao                  object
recommend_to_a_friend     object
site_category_lv1         object
reviewer_birth_year      float64
reviewer_gender           object
overall_rating             int64
dtype: object

##### Task 5: Dividir o Conjunto de Dados em Treino, Teste e Validação

Divisão entre Treino e Teste

In [10]:
data.head()

,reviewer_id,positivo,negativo,review,anotacao,recommend_to_a_friend,site_category_lv1,reviewer_birth_year,reviewer_gender,overall_rating
0,c2b3ebfc9238e1462555a7e36539a078c6fb85e5438e39...,1.0,1.0,agilidade configuração produto bom imagem som ...,positivo negativo,Yes,Informática,1899.0,M,3
1,1345ab31c9fa498aac8dd78b1c1069a056f237e20fdba7...,1.0,1.0,atender expectativa outro fritadeira outro mar...,positivo negativo,Yes,Eletroportáteis,1900.0,F,3
2,b68330385cc203b6e8249f796d592ae6173df1adda5ae3...,1.0,1.0,bom filme filme assistir curtir famílio luta v...,positivo negativo,Yes,Filmes e Séries,1906.0,M,3
3,ed921e3d758d33b09938bb46d932c3cdbef9f821b68a0e...,1.0,0.0,produto bom acabamento recomendar produto pois...,positivo,Yes,Móveis,1908.0,M,3
4,dd8279ea157b3802bb5b68a2fb647595178e941f889386...,1.0,0.0,preço ta otimo calça real show bola bom econom...,positivo,Yes,Moda,1924.0,F,3


In [11]:
#Define as Variáveis Target e Predictor - BOW
X1 = X_bow
y = data[['positivo', 'negativo']]

In [12]:
# Divide os dados em treino e teste - BOW
X_train,X_test,y_train,y_test = train_test_split(X1,y,test_size=0.2,random_state=42)

In [13]:
#Define as Variáveis Target e Predictor - TD-IDF
X2 = X_tfidf
y = data[['positivo', 'negativo']]

In [14]:
# Divide os dados em treino e teste - TD-IDF
X_train,X_test,y_train,y_test = train_test_split(X2,y,test_size=0.2,random_state=42)